## Метод Гауса

In [17]:
import numpy as np
import copy
import random

##### Введіть, будь ласка, матрицю A системи  порядково, використовуючи space як розділювач, і вектор b

In [18]:
# input the data from keyboard
error = True
while error:
    try:
        rows_number = int(input('Input the number of rows:'))
        columns_number = int(input('Input the number of columns:'))
        matrix_size = [rows_number, columns_number]
        print('\n' + 'The size of matrix is ' + str(rows_number) + ' x ' + str(columns_number) + '\n')
        
        matrix = []
        while (rows_number != 0):
            row = input('Input the row of the matrix as a separator use space:')
            row = row.split(' ')
            row = [float(x) for x in row]
            matrix.append(row)
            rows_number = rows_number - 1
        
        vector_b = []
        row = input('Input the vector b as a separator use space:')
        row = row.split(' ')
        vector_b = [float(x) for x in row]
        error = False
        
        print('\n' + 'The matrix A is:')
        print(np.array(matrix))
        print('\n' + 'The vector b is:')
        print(np.array(vector_b))
        
    except ValueError:
        error = True
        print("Incorrent format. Try again")
    

KeyboardInterrupt: 

#### Приклади з варіанту №4

In [19]:
A_1 = [[2, 2, -1, 1], [-3, 0, 3, 0], [-1, 3, 3, 2], [1, 0, 0, 4]]
b_1 = [3, -9, -7, 4]

A_2 = [[0, 3, -1], [9, 24, 1], [21, -1, -16]]
b_2 = [7, 20, 63]

A_3 = [[5, 0, -7, 0], [-1, 6, 0, 1], [2, -6, -4, -5], [-6, -6, 15, 7]]
b_3 = [-123, 60, -108, 159]

#### Допоміжні функції

In [4]:
def switch_rows(matrix, i, j): # switch the i row to j row
    k = matrix[i]
    matrix[i] = matrix[j]
    matrix[j] = k
    return matrix

def find_main_element(matrix, j): # return the main element and number of row where it is
    main_element = matrix[j][j]
    row_number = j
    for i in range(j+1, len(matrix)):
        if abs(main_element) < abs(matrix[i][j]):
            main_element = matrix[i][j]
            row_number = i
    return main_element, row_number

def count_determinant(triangular_matrix, permutations): 
    determinant = (-1) ** permutations
    for i in range(len(triangular_matrix)):
        determinant = determinant * triangular_matrix[i][i]
    return determinant 

def count_norm_matrix(matrix): # Октаєдрична норма
    max_sum = 0
    for j in range(len(matrix)):
        sum_column = 0
        for i in range(len(matrix)):
            sum_column += abs(matrix[i][j])
        if (max_sum < sum_column):
            max_sum = sum_column
    return max_sum

def move_up(under_triangular_matrix, vector_b):
    #error = False
    size =len(under_triangular_matrix)
    x = np.zeros(size)
    if (under_triangular_matrix[size - 1][size - 1] != 0): 
        x[size - 1] = vector_b[size - 1] / under_triangular_matrix[size - 1][size - 1]
        i = size - 2
        while(i >= 0):
            s = 0
            for j in range(i+1, size):
                s += under_triangular_matrix[i][j] * x[j]
            x[i] = (vector_b[i] - s) / under_triangular_matrix[i][i]
            i = i - 1
    #else:
    #    error = True
    return x

def move_down(down_triangular_matrix, vector_b):
    #error = False
    size =len(down_triangular_matrix)
    x = np.zeros(size)
    if (down_triangular_matrix[0][0] != 0):
        x[0] = vector_b[0] / down_triangular_matrix[0][0]
        i = 1
        while (i < size):
            s = 0
            for j in range(i):
                s += down_triangular_matrix[i][j] * x[j]
            x[i] = (vector_b[i] - s) / down_triangular_matrix[i][i]
            i = i + 1
    #else:
    #    error = True
    return x

In [5]:
# solution fo the system with Gauss` method
def Gauss(matrix, vector_b): 
    error = False
    permutations = 0
    for j in range(len(matrix) - 1):
        main_element, row_number  = find_main_element(matrix, j)
        #print(main_element)
        if (main_element == 0): 
            error = True
            break
        else:
            if (row_number != j):
                matrix = switch_rows(matrix, j, row_number)
                vector_b = switch_rows(vector_b, j, row_number)
                permutations += 1
            for i in range(j+1, len(matrix)):
                t = matrix[i][j] / main_element
                vector_b[i] = vector_b[i] - t * vector_b[j]
                #print(vector_b)
                matrix[i][j] = 0
                for k in range(j+1, len(matrix)):
                    matrix[i][k] = matrix[i][k] - t * matrix[j][k]
                #print(matrix) 
    if not error:
        x = np.zeros(len(matrix))
        if (matrix[len(matrix) - 1][len(matrix) - 1] != 0): # вище іфа перевірити чи не нуль
            x[len(matrix) - 1] = vector_b[len(matrix) - 1] / matrix[len(matrix) - 1][len(matrix) - 1]
            i = len(matrix) - 2
            while(i >= 0):
                s = 0
                for j in range(i+1, len(matrix)):
                    s += matrix[i][j] * x[j]
                x[i] = (vector_b[i] - s) / matrix[i][i]
                i = i - 1
            determinant = count_determinant(matrix, permutations)
            return matrix, determinant, x, error, row_number
        else:
            error = True
 
    return matrix, 0, 0, error, row_number


In [6]:
# LU factorization

def factorizate_matrix(matrix):
    error = False
    L_matrix = np.zeros_like(matrix, dtype = float)
    U_matrix = np.zeros_like(matrix, dtype = float)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if (i == j):
                L_matrix[i][i] = 1
                                                       
            if (i <= j):
                sum_1 = 0
                for k in range(i):
                    sum_1 += L_matrix[i][k] * U_matrix[k][j]
                U_matrix[i][j] = matrix[i][j] - sum_1
            
            if (i > j):
                sum_2 = 0
                for n in range(j):
                    sum_2 += L_matrix[i][n] * U_matrix[n][j]
                if (U_matrix[j][j] != 0):    
                    L_matrix[i][j] = (1 / U_matrix[j][j]) * (matrix[i][j] - sum_2)
                else:
                    error = True
                    break

    return L_matrix, U_matrix, error

In [7]:
# reverse matrix based on LU factorisation

def find_reverse_matrix(L_matrix, U_matrix):
    reverse_matrix = np.zeros_like(L_matrix, dtype = float)
    y = np.zeros(len(L_matrix))
    x = np.zeros(len(L_matrix))
    
    for j in range(len(U_matrix)):
        one_vector = np.zeros(len(L_matrix))
        y = np.zeros(len(L_matrix))
        x = np.zeros(len(L_matrix))
        one_vector[j] = 1
        y = move_down(L_matrix, one_vector)
        x = move_up(U_matrix, y)
        
        for k in range(len(x)):
            reverse_matrix[k][j] = x[k]
        
    return reverse_matrix

In [8]:
def print_answers(A, b):
    initial_A = copy.deepcopy(A)
    initial_b = copy.deepcopy(b)
    matrix_A, det, x, error_Gauss, row = Gauss(A, b)
    L, U, error_LU = factorizate_matrix(initial_A)
    rev_A = find_reverse_matrix(L, U)
    cond_A = count_norm_matrix(initial_A) * count_norm_matrix(rev_A)
    
    print('The initial matrix A and the vector b')
    print(initial_A)
    print(initial_b)

    if error_Gauss:
        print('The determinant of the matrix A is 0')
        print('\n' + 'Zero main element occured in the row #' + str(row))
        print('\n' + 'Answer of libraries')
        print('The determinant of the matrix A: ')
        print(np.linalg.det(initial_A))
        print('\n' + 'The matrix can not be factorized in L and U')
        print('\n' + 'The reverse matrix does not exist')
    else:
        print('The determinant of the matrix A:')
        print(det)
        print('The solution of the system:')
        print(x)
    
        initial_A = np.array(initial_A)
        initial_b = np.array(initial_b)
        print('\n' + 'Answers of libraries')
        print('The determinant of the matrix A: ')
        print(np.linalg.det(initial_A))
        print('\n' + 'The solution of the system:' )
        print(np.linalg.solve(initial_A, initial_b))
    
        print('\n' + 'The error of solution:')
        print(initial_A.dot(x) - initial_b)
    
        if not error_LU:
            print('\n' + 'The matrix L')
            print(L)
            print('\n' + 'The matrix U')
            print(U)
            print('LU - A')
            print(L.dot(U) - initial_A)
            print('\n' + 'The reverse matrix')
            print(rev_A)
            print('\n' + 'The product of the matrix and the reverse matrix')
            print(rev_A.dot(initial_A))
            print('\n' + 'The conditional number')
            print(cond_A)
        else:
            print('\n' + 'The matrix can not be factorized in L and U because one of minor is zero')
    
    

#### Результати для матриці 1 з варіанту №4

In [9]:
print_answers(A_1, b_1)

The initial matrix A and the vector b
[[2, 2, -1, 1], [-3, 0, 3, 0], [-1, 3, 3, 2], [1, 0, 0, 4]]
[3, -9, -7, 4]
The determinant of the matrix A:
8.999999999999998
The solution of the system:
[ 4.00000000e+00 -2.00000000e+00  1.00000000e+00 -2.22044605e-16]

Answers of libraries
The determinant of the matrix A: 
8.999999999999998

The solution of the system:
[ 4.00000000e+00 -2.00000000e+00  1.00000000e+00 -2.22044605e-16]

The error of solution:
[ 0.0000000e+00  0.0000000e+00 -8.8817842e-16  0.0000000e+00]

The matrix L
[[ 1.          0.          0.          0.        ]
 [-1.5         1.          0.          0.        ]
 [-0.5         1.33333333  1.          0.        ]
 [ 0.5        -0.33333333  2.          1.        ]]

The matrix U
[[ 2.   2.  -1.   1. ]
 [ 0.   3.   1.5  1.5]
 [ 0.   0.   0.5  0.5]
 [ 0.   0.   0.   3. ]]
LU - A
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e

#### Результати для матриці 2 з варіанту №4

In [10]:
print_answers(A_2, b_2) # перший мінор нульовий, тому не можу факторизувати

The initial matrix A and the vector b
[[0, 3, -1], [9, 24, 1], [21, -1, -16]]
[7, 20, 63]
The determinant of the matrix A:
1008.0
The solution of the system:
[ 0.  1. -4.]

Answers of libraries
The determinant of the matrix A: 
1008.0000000000002

The solution of the system:
[ 0.  1. -4.]

The error of solution:
[0. 0. 0.]

The matrix can not be factorized in L and U because one of minor is zero


/home/khrystyna/.local/lib/python3.5/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in double_scalars
/home/khrystyna/.local/lib/python3.5/site-packages/ipykernel_launcher.py:60: RuntimeWarning: divide by zero encountered in double_scalars
/home/khrystyna/.local/lib/python3.5/site-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


####  Результати для матриці 3 з варіанту №4

In [11]:
print_answers(A_3, b_3)

The initial matrix A and the vector b
[[5, 0, -7, 0], [-1, 6, 0, 1], [2, -6, -4, -5], [-6, -6, 15, 7]]
[-123, 60, -108, 159]
The determinant of the matrix A is 0

Zero main element occured in the row #3

Answer of libraries
The determinant of the matrix A: 
-2.0783375020982918e-13

The matrix can not be factorized in L and U

The reverse matrix does not exist


#### Результати для матриці й вектора введених з клавіатури

In [12]:
print_answers(matrix, vector_b)

The initial matrix A and the vector b
[[1.0, 2.0, 3.0], [5.0, 7.0, 2.0], [1.0, 10.0, -9.0]]
[1.0, -7.0, 0.0]
The determinant of the matrix A:
140.0
The solution of the system:
[-2.99285714  0.93571429  0.70714286]

Answers of libraries
The determinant of the matrix A: 
139.99999999999994

The solution of the system:
[-2.99285714  0.93571429  0.70714286]

The error of solution:
[8.8817842e-16 0.0000000e+00 8.8817842e-16]

The matrix L
[[ 1.          0.          0.        ]
 [ 5.          1.          0.        ]
 [ 1.         -2.66666667  1.        ]]

The matrix U
[[  1.           2.           3.        ]
 [  0.          -3.         -13.        ]
 [  0.           0.         -46.66666667]]
LU - A
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

The reverse matrix
[[-0.59285714  0.34285714 -0.12142857]
 [ 0.33571429 -0.08571429  0.09285714]
 [ 0.30714286 -0.05714286 -0.02142857]]

The product of the matrix and the reverse matrix
[[ 1.00000000e+00 -4.44089210e-16 -2.22044605e-16]
 [ 2.08166817e-16  1

#### Збурення вектора вільних членів

In [15]:
def perturbation(b, number_coordinates):
    permutation = np.random.permutation(range(len(b)))
    for k in range(number_coordinates):
        b[permutation[k]] += 0.01 * b[permutation[k]]
    return b
    

#### Порівняння для системи 1 збуреного й незбуреного розв'язку

In [16]:
def compare_solutions(A, b):
    initial_A = copy.deepcopy(A)
    initial_b = copy.deepcopy(b)
    new_b = perturbation(b, number_coordinates=1)
    
    matrix_A_1, det, x_1, error_Gauss, row = Gauss(initial_A, initial_b)
    matrix_A_2, det, x_2, error_Gauss, row = Gauss(A, new_b)
    print('The solution without perturbation')
    print(x_1)
    print('\n' + 'The solution with perturbation 1%')
    print(x_2)
    print('\n'+ 'Absolute difference:')
    print(abs(x_1 - x_2))
    

In [20]:
compare_solutions(A_1, b_1)

The solution without perturbation
[ 4.00000000e+00 -2.00000000e+00  1.00000000e+00 -2.22044605e-16]

The solution with perturbation 1%
[ 3.81333333 -1.93        0.81333333  0.04666667]

Absolute difference:
[0.18666667 0.07       0.18666667 0.04666667]
